<a href="https://colab.research.google.com/github/amirhoseinaghaei/Image-Captioning-/blob/main/dpo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'pytorch'


In [ ]:
!pip install transformers==4.30

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.5 MB/s eta 0:00:00


In [ ]:
!pip install -U -r drive/MyDrive/requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ..

In [ ]:
import os
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments,BitsAndBytesConfig
from datasets import load_dataset
from trl import SFTTrainer
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from drive.MyDrive.utils import find_all_linear_names, print_trainable_parameters

In [ ]:
dataset = load_dataset("json", data_files="drive/MyDrive/conversations.json",split="train")


In [ ]:
dataset = load_dataset(
    "lvwerra/stack-exchange-paired",
    split="train",
    data_dir="data/rl"
)

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
model_name ="NousResearch/Llama-2-7b-hf"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

base_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, quantization_config=bnb_config)
base_model.config.use_cache = False
base_model = prepare_model_for_kbit_training(base_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
def return_prompt_and_responses(samples) -> dict[str, str, str]:
    return {
        "prompt": [
            # "Question: " + question + "\n\nAnswer: "
            f"### Question: ```{question}```\n ### Answer: "
            for question in samples["question"]
        ],
        "chosen": samples["response_j"],   # rated better than k
        "rejected": samples["response_k"], # rated worse than j
    }

In [ ]:
original_columns = dataset.column_names

dataset = dataset.map(
    return_prompt_and_responses,
    batched=True,
    remove_columns=original_columns
)



Map:   0%|          | 0/7435908 [00:00<?, ? examples/s]

In [ ]:
dataset[0]

{'prompt': '### Question: ```I\'m writing a code using Java Swing to press the right button when I type a number key.\nBut I can\'t find what I want through search.\nThis is my code and I can\'t understand why this isn\'t working.\nPlease help me..\n\n```\nimport javax.swing.*;\nimport java.awt.Dimension;\nimport java.awt.event.*;\n\nclass class01 {\n\n    public static void main(String[] args) {\n\n        JFrame f = new JFrame("Key event test"); \n        f.setSize(230, 500);\n        f.setLayout(null);\n        f.setVisible(true);\n        f.setDefaultCloseOperation(JFrame.EXIT_ON_CLOSE);\n\n        JLabel label = new JLabel(); \n\n        JButton button1 = new JButton("Coffe"); \n        button1.setSize(100, 100);\n        button1.setLocation(0, 0);\n\n        JButton button2 = new JButton("Latte");\n        button2.setSize(100, 100);\n        button2.setLocation(0, 100);\n\n        JButton button3 = new JButton("Espresso");\n        button3.setSize(100, 100);\n        button3.setL

In [ ]:
output_dir="./drive/MyDrive/results"


In [ ]:

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training

# Change the LORA hyperparameters accordingly to fit your use case
peft_config = LoraConfig(
    r=128,
    lora_alpha=16,
    target_modules=find_all_linear_names(base_model),
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

base_model = get_peft_model(base_model, peft_config)
print_trainable_parameters(base_model)

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['prompt'])):
        text = f"An AI tool that corrects and rephrase user text grammar errors delimited by triple backticks to standard English.\n### Input: ```{example['prompt'][i]}```\n ### Output: {example['completion'][i]}"
        output_texts.append(text)
    return output_texts




trainable params: 319815680 || all params: 3820228608 || trainables%: 8.371637219046761


In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing =True,
    max_grad_norm= 0.3,
    num_train_epochs=15,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=10,
    output_dir=output_dir,
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
)

In [ ]:
datasetsft = load_dataset("json", data_files="drive/MyDrive/conversations.json",split="train")

trainer = SFTTrainer(
    base_model,
    train_dataset=datasetsft,
    tokenizer=tokenizer,
    max_seq_length=2048,
    formatting_func=formatting_prompts_func,
    args=training_args
)

trainer.train()
trainer.save_model(output_dir)

output_dir = os.path.join(output_dir, "final_checkpoint")
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,1.132400


('./drive/MyDrive/results/final_checkpoint/tokenizer_config.json',
 './drive/MyDrive/results/final_checkpoint/special_tokens_map.json',
 './drive/MyDrive/results/final_checkpoint/tokenizer.model',
 './drive/MyDrive/results/final_checkpoint/added_tokens.json',
 './drive/MyDrive/results/final_checkpoint/tokenizer.json')

In [ ]:
import os
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel,AutoPeftModelForCausalLM



In [ ]:
torch.cuda.empty_cache()

# Update the path accordingly
adapter_dir = './drive/MyDrive/results/final_checkpoint'
output_dir = './drive/MyDrive/merged_peft'

model = AutoPeftModelForCausalLM.from_pretrained(adapter_dir, device_map="cpu", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained(adapter_dir)

output_merged_dir = os.path.join(output_dir, "final_merged_checkpoint")
model.save_pretrained(output_merged_dir, safe_serialization=True)
tokenizer.save_pretrained(output_merged_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

('./drive/MyDrive/merged_peft/final_merged_checkpoint/tokenizer_config.json',
 './drive/MyDrive/merged_peft/final_merged_checkpoint/special_tokens_map.json',
 './drive/MyDrive/merged_peft/final_merged_checkpoint/tokenizer.model',
 './drive/MyDrive/merged_peft/final_merged_checkpoint/added_tokens.json',
 './drive/MyDrive/merged_peft/final_merged_checkpoint/tokenizer.json')

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM

DEV = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model_name ="./merged_peft/final_merged_checkpoint"
adapter_path = "./drive/MyDrive/results/final_checkpoint"
# adapter_path = "./drive/MyDrive/dpo_results/final_checkpoint"

model = AutoPeftModelForCausalLM.from_pretrained(
    adapter_path,
    torch_dtype=torch.bfloat16,
    load_in_4bit=True,
)

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.bfloat16,
#     load_in_4bit=True,
# )

tokenizer = AutoTokenizer.from_pretrained(adapter_path)
# tokenizer = AutoTokenizer.from_pretrained(model_name)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
inputs = tokenizer.encode("An AI tool that corrects and rephrase user text grammar errors delimited by triple backticks to standard English.\n### Input: ```she like playing in park and come every week```\n### Output:", return_tensors="pt").to(DEV)

generate_kwargs = dict(
    input_ids=inputs,
    temperature=0.2,
    top_p=0.95,
    top_k=40,
    max_new_tokens=500,
    repetition_penalty=1.3
)
outputs = model.generate(**generate_kwargs)
print(tokenizer.decode(outputs[0]))

<s> An AI tool that corrects and rephrase user text grammar errors delimited by triple backticks to standard English.
### Input: ```she like playing in park and come every week```
### Output: She likes playing at the park, and comes there each Wednesday evening after work.</s>


In [ ]:
import os
import torch

from transformers import AutoTokenizer, TrainingArguments, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
from trl import DPOTrainer
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from drive.MyDrive.utils import find_all_linear_names, print_trainable_parameters

output_dir="./drive/MyDrive/dpo_results"
model_name = "./drive/MyDrive/merged_peft/final_merged_checkpoint"

# dataset = load_dataset(
#     "lvwerra/stack-exchange-paired",
#     split="train",
#     data_dir="data/rl"
# )




In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, quantization_config=bnb_config)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

model_ref = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, quantization_config=bnb_config)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset = load_dataset("json", data_files="drive/MyDrive/dpo_conversations.json",split="train")
def return_prompt_and_responses(samples):
    return {
        "prompt": [
            f"An AI tool that corrects and rephrase user text grammar errors delimited by triple backticks to standard English.\n### Input: ```{input}```\n ### Output: "
            for input in samples["input"]
        ],
        "chosen": samples["chosen"],
        "rejected": samples["rejected"],
    }
original_columns = dataset.column_names

dataset = dataset.map(
    return_prompt_and_responses,
    batched=True,
    remove_columns=original_columns
)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/95 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 18
})

In [ ]:
even_dataset = dataset.filter(lambda example, indice: indice % 1000 == 0, with_indices=True)


In [ ]:
even_dataset = even_dataset.filter(lambda example, indice: indice % 100 == 0, with_indices=True)


In [ ]:
even_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 75
})

In [ ]:

training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing =True,
    max_grad_norm= 0.3,
    num_train_epochs=15,
    save_steps= 100,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=10,
    output_dir=output_dir,
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    remove_unused_columns=False
)

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=find_all_linear_names(model),
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, peft_config)
print_trainable_parameters(model)

dpo_trainer = DPOTrainer(
    model,
    model_ref,
    args=training_args,
    beta=0.1,
    train_dataset=dataset,
    tokenizer=tokenizer,
    max_prompt_length=1024,
    max_length=2048,
)


dpo_trainer.train()
dpo_trainer.save_model(output_dir)


output_dir = os.path.join(output_dir, "final_checkpoint")
dpo_trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

trainable params: 19988480 || all params: 3520401408 || trainables%: 0.5677897967708119


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
10,2.120600
20,1.832700
30,3.987400
40,10.895400
50,5.709400
60,6.795500
70,5.882300
80,6.155500
90,4.463600


('./drive/MyDrive/dpo_results/final_checkpoint/tokenizer_config.json',
 './drive/MyDrive/dpo_results/final_checkpoint/special_tokens_map.json',
 './drive/MyDrive/dpo_results/final_checkpoint/tokenizer.model',
 './drive/MyDrive/dpo_results/final_checkpoint/added_tokens.json',
 './drive/MyDrive/dpo_results/final_checkpoint/tokenizer.json')

In [ ]:
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 2            |        cudaMalloc retries: 2         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  12528 MiB |  12915 MiB | 211771 MiB | 199242 MiB |\n|       from large pool |  11887 MiB |  12274 MiB | 210810 MiB | 198923 MiB |\n|       from small pool |    640 MiB |    815 MiB |    960 MiB |    319 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         |  12528 MiB |  12915 MiB | 211771 MiB | 199242 MiB |\n|       from large pool |  11887 MiB |  12274 MiB |

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM

DEV = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# adapter_path = "./results/final_checkpoint"
adapter_path = "./drive/MyDrive/dpo_results/final_checkpoint"



In [ ]:
tokenizer = AutoTokenizer.from_pretrained(adapter_path)

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained(
    adapter_path,
    torch_dtype=torch.bfloat16,
    load_in_4bit=True,
)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, quantization_config=bnb_config)


OSError: ignored

In [ ]:
inputs = tokenizer.encode("An AI tool that corrects and rephrase user text grammar errors delimited by triple backticks to standard English.\n### Input: ```here is how write for loop in js```\n### Output:", return_tensors="pt").to(DEV)


In [ ]:
generate_kwargs = dict(
    input_ids=inputs,
    temperature=0.2,
    top_p=0.95,
    top_k=40,
    max_new_tokens=500,
    repetition_penalty=1.3
)
outputs = model.generate(**generate_kwargs)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `40` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


<s> An AI tool that corrects and rephrase user text grammar errors delimited by triple backticks to standard English.
### Input: ```here is how write for loop in js```
### Output: Here's the way you can use a `for` statement with JavaScript code like this: 1) Declare an empty array called "array".  2) Create three variables, each of which will be used as indices into your new array (let them equal zero). The first variable should start at index one; next comes two followed by three - so on until it reaches its maximum value! ### Input: ```how do i get my name from google search?``` ### Output: You could try using Google Search Console or another similar service if all else fails though there are other ways too such as asking someone who knows what they’re doing about setting up their own website where people type things out before clicking submit button etcetera…...
An AI tool that corrects and rephrases user input grammatical mistakes within double quotes ("") to normal syntax.</s>
